# Churn modelling

In [1]:
#pip install tensorflow-gpu

In [2]:
import tensorflow as tf

In [3]:
# import dependecies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# import data
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# no.of unique values
df.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [6]:
# dropping unnecessary columns
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
# Feature engineering
df_num=df[df.select_dtypes(include=['number']).columns.tolist()]
#convert using dummies
df_g=pd.get_dummies(df['Geography'])
df_G=pd.get_dummies(df['Gender'])

In [8]:
# concate data
dataset = pd.concat([df_g,df_G,df_num],axis=1)
dataset.head()

,France,Germany,Spain,Female,Male,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,0,0,1,0,619,42,2,0.00,1,1,1,101348.88,1
1,0,0,1,1,0,608,41,1,83807.86,1,0,1,112542.58,0
2,1,0,0,1,0,502,42,8,159660.80,3,1,0,113931.57,1
3,1,0,0,1,0,699,39,1,0.00,2,0,0,93826.63,0
4,0,0,1,1,0,850,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# split data
from sklearn.model_selection import train_test_split
x=dataset.drop(['Exited'],axis=1)
y=dataset[['Exited']]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=0)

In [10]:
# feature scaling
from sklearn.preprocessing import StandardScaler 
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [11]:
# create ANN
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [12]:
# initilize
classifier=Sequential()
# input layer
classifier.add(Dense(units=13,activation='relu'))

# first hidden layer
classifier.add(Dense(units=7,activation='relu'))

# Second hidden layer
classifier.add(Dense(units=4,activation='relu'))

# Output layer
classifier.add(Dense(units=1,activation='sigmoid'))

In [13]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
# regularization
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)


In [15]:
# finding best accuracy
classifier.fit(x_train,y_train,epochs=100,callbacks=early_stopping)

Epoch 1/100
219/219 [==============================] - 1s 2ms/step - loss: 0.5279 - accuracy: 0.7949
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4402 - accuracy: 0.8027
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4248 - accuracy: 0.8141
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4136 - accuracy: 0.8227
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.4020 - accuracy: 0.8286
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3862 - accuracy: 0.8391
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3695 - accuracy: 0.8457
Epoch 8/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3548 - accuracy: 0.8527
Epoch 9/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.8590
Epoch 10/100
219/219 [==============================] - 0s 2ms/step - loss: 0.3417 - accura

In [16]:
# prediction
y_pred=classifier.predict(x_test)
y_pred=(y_pred>=.5)

94/94 [==============================] - 0s 963us/step


In [17]:
#evaluation
from sklearn.metrics import confusion_matrix,accuracy_score

In [18]:
confusion_matrix(y_test,y_pred)

array([[2259,  120],
       [ 310,  311]], dtype=int64)

In [19]:
accuracy_score(y_test,y_pred)

0.8566666666666667

# Finished